In [2]:
from shapely.geometry import Point, Polygon, shape
from shapely import ops
from streamlit_folium import folium_static
import os
import pandas as pd
import folium
import json
import geopandas as gpd
from json import loads
import re
import numpy as np
import fuzzywuzzy

C:\Users\belendelrociogarcia\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
!curl -L -X GET --header 'Accept: application/json' 'https://datos.madrid.es/egob/catalogo/202584-0-aparcamientos-residentes.json'

In [ ]:
import requests

url = 'https://datos.madrid.es/egob/catalogo/202584-0-aparcamientos-residentes.json'
headers = {'Accept': 'application/json'}

response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Save the raw JSON text to a file
    with open('aparcamientos_data.json', 'w', encoding='utf-8') as json_file:
        json_file.write(response.text)

    print("JSON data has been exported to 'aparcamientos_data.json'")
else:
    print(f"Error in the request. Status code: {response.status_code}")
    print("Response content:")
    print(response.text)

In [3]:
# Specify the path to the file
file_path = 'aparcamientos_data.json'

# Read the JSON content
with open(file_path, 'r', encoding='utf-8') as json_file:
    file_content = json_file.read()

# Check if the file content is not empty
if file_content:
    # Attempt to load the JSON content into a Pandas DataFrame
    try:
        # Remove trailing commas (if any) before parsing as JSON
        cleaned_content = file_content.rstrip(',')
        data = json.loads(cleaned_content)

        # Convert the JSON data to a Pandas DataFrame
        df = pd.json_normalize(data.get('@graph', []))

        # Extract and clean the desired fields
        parking_data = df[[
            'id', 'title',
            'address.locality', 'address.district.@id','address.area.@id', 'address.street-address',
            'location.latitude', 'location.longitude',
            'organization.organization-desc',  'organization.accesibility'
        ]].copy()

        # Rename columns for clarity
        parking_data.columns = [
            'id', 'title', 'locality', 'district', 'area', 'street_address', 'latitude', 'longitude', 'organization', 'accesibility'
        ]

        # Extract 'Plazas' information using regular expressions
        import re

        def extract_parking_info(description):
            # Pattern for cases with explicit counts for public and resident spaces
            pattern1 = re.compile(r'Plazas:\s*(\d+)\s*públicas\s*y\s*(\d+)\s*para\s*residentes')

            # Pattern for cases with a general count of spaces
            pattern2 = re.compile(r'Plazas:\s*(\d+)')

            # Check for the first pattern
            match1 = pattern1.search(description)
            if match1:
                public_spaces = int(match1.group(1))
                resident_spaces = int(match1.group(2))
                general_spaces = public_spaces + resident_spaces
                return public_spaces, resident_spaces, general_spaces

            # Check for the second pattern
            match2 = pattern2.search(description)
            if match2:
                general_spaces = int(match2.group(1))
                return np.nan, np.nan, general_spaces

            # If no match, return zeros for all counts
            return 0, 0, 0
        # Apply the function to create new columns for different types of spaces
        parking_data[['public_spaces', 'resident_spaces', 'general_spaces']] = parking_data['organization'].apply(extract_parking_info).apply(pd.Series)
        
        # Define a function to apply the regex substitution
        def add_space_to_camel_case(text):
            return re.sub(r'([a-z])([A-Z])', r'\1 \2', text.split("/")[-1])

        # Create new columns for 'Plazas Publicas' and 'Plazas para Residentes'
#         parking_data['plazas_publicas'] = plaza_info[0].astype(float)
#         parking_data['plazas_residentes'] = plaza_info[1].astype(float)
        parking_data['district'] = parking_data["district"].str.split("/").str[-1]
        parking_data['area'] = parking_data["area"].apply(add_space_to_camel_case)
        
        # Print the cleaned data
        print(parking_data)
        
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
else:
    print("File is empty.")

           id                                              title locality  \
0    11544480                      Aparcamiento mixto. Alcántara   MADRID   
1       13470              Aparcamiento mixto. Arquitecto Ribera   MADRID   
2       59060  Aparcamiento mixto. Avenida de América (interc...   MADRID   
3     5898885            Aparcamiento mixto. Avenida de Portugal   MADRID   
4       13475                         Aparcamiento mixto. Brasil   MADRID   
..        ...                                                ...      ...   
268     13614         Aparcamiento para residentes. Vista Alegre   MADRID   
269     13493            Aparcamiento para residentes. Vital Aza   MADRID   
270     61831        Aparcamiento para residentes. Vía Carpetana   MADRID   
271     13514        Aparcamiento para residentes. Ángel Carbajo   MADRID   
272     13463              Aparcamiento público. Plaza de España   MADRID   

         district            area                street_address   latitude 

Hay 4 aparcamientos que no siguen el patrón observado. Eliniminamos las observaciones con missing values en 'organization' y tratamos el par de parkings restantes de los que podemos extraer información útil. En el parking con índice 186 han escrito 'Plazas Totales: ' en lugar de 'Plazas: '. En la observación 137 han escrito 'Plazas : ' por eso introducimos también el valor manualmente .

In [4]:
parking_data[parking_data['public_spaces'] == 0]

,id,title,locality,district,area,street_address,latitude,longitude,organization,accesibility,public_spaces,resident_spaces,general_spaces
23,11544439,Aparcamiento mixto. Plaza de Santo Domingo,MADRID,Centro,Palacio,PLAZA SANTO DOMINGO,40.420133,-3.708260,"Plazas Totales: 320 plazas, Plazas motos: 16.",0,0.0,0.0,0.0
102,13535,Aparcamiento para residentes. Darwin,MADRID,Moratalaz,Fontarron,AVENIDA MORATALAZ 58,40.405639,-3.653755,,0,0.0,0.0,0.0
137,5421792,Aparcamiento para residentes. Ilíada,MADRID,SanBlas-Canillejas,Canillejas,CALLE LA ILIADA 1,40.435583,-3.608601,Plazas : 333 Abierto 24 Horas Titularidad: A...,0,0.0,0.0,0.0
186,13640,Aparcamiento para residentes. Ofelia Nieto,MADRID,Moncloa-Aravaca,Valdezarza,CALLE OFELIA NIETO 55,40.461344,-3.709415,,0,0.0,0.0,0.0


In [16]:
parking_data.loc[23, 'general_spaces'] = 320
parking_data.loc[137, 'general_spaces'] = 333
parking_data = parking_data[parking_data['organization'] != '\xa0\xa0']
parking_data.loc[parking_data['area'] == 'Casco HVallecas', 'area'] = 'Casco Histórico de Vallecas'
parking_data.loc[parking_data['area'] == 'Casco HVicalvaro', 'area'] = 'Casco histórico de Vicálvaro'

### MAPA

In [6]:
os.getcwd()

'C:\\Users\\belendelrociogarcia\\Downloads\\mdsafiescuela\\aparcamientos'

In [7]:
barrios = gpd.read_file('Barrios.zip')

### Choropleth Maps

Agrupamos los parkings por barrio

In [17]:
areas = list(barrios['NOMBRE'].values)
barrios[barrios['NOMBRE'].isin(total_parking['area'])].shape[0]

61

In [18]:
from fuzzywuzzy import process

def parking_areas():
    total_parking = parking_data.groupby('area').sum('general_spaces').sort_values('general_spaces', ascending=False).reset_index()[['area','general_spaces']]
    
    parking_areas = list(total_parking['area'])

    # Create a dictionary to store the matches
    matches = {}

    # Iterate through items in list1
    for i, item1 in enumerate(parking_areas):
        # Use process.extractOne to find the best match in list2 for each item in list1
        match, score = process.extractOne(item1, areas)

        # If the matching score is above a certain threshold (adjust as needed)
        if score >= 40:
            matches[item1] = match
            # Replace the value in list1 with the matched value from list2
            parking_areas[i] = match

    # Updated list1 with matched values
    return parking_areas

In [19]:
barrios[barrios['NOMBRE'].isin(parking_areas())].shape[0]

88

In [20]:
# Assuming 'total_parking' is your DataFrame with parking information
filtered_barrios = barrios[barrios['NOMBRE'].isin(parking_areas())]

In [21]:
mapa = folium.Map(location=[40.4309, -3.6878], zoom_start=11, scrollWheelZoom=True, tiles='CartoDB positron')

tooltip = "Haz click para obtener más información"

# Replace 'path/to/parking-icon.png' with the actual path to your parking icon image
icon_path = 'https://fontawesome.com/v5/icons/parking?f=classic&s=regular'

for i, parking in parking_data.iterrows():
    coordinates = [parking['latitude'], parking['longitude']]
    popup_content = f"Name: {parking['title']}<br>Capacity: {parking['general_spaces']}"
    tooltip = tooltip
    # Add a marker for each parking
    folium.Marker(location=coordinates, popup=popup_content).add_to(mapa)

# Marker for Afi Escuela
marker = folium.Marker(
    location=[40.4309, -3.6878],
    # icon=folium.CustomIcon(icon_image=icon_path, icon_size=(30, 30)),
    popup="<strong>Afi Escuela</strong>",
    icon=folium.CustomIcon(icon_image=icon_path, icon_size=(30, 30)),
    tooltip=tooltip
)

marker.add_to(mapa)

# Create the choropleth layer using the filtered GeoJSON data
folium.Choropleth(
    geo_data=filtered_barrios,
    name='choropleth',
    data=total_parking,
    columns=['area', 'general_spaces'],
    key_on='feature.properties.NOMBRE',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2
).add_to(mapa)

folium.GeoJson(filtered_barrios, aliases=['area: '], localize=True, labels=True, parse_html=False).add_to(mapa)

# Display the map
mapa

Algunos de los parkings quedan fuera de los barrios en los que hay parkings. Vamos a estudiar cada caso.

## Casco HVallecas

In [ ]:
parking_data[parking_data['title'].str.contains('Altos de Lillo')]

Buscamos un barrio en la lista de los barrios que contenga la subcadena 'Vallecas'

In [ ]:
barrios[barrios['NOMBRE'].str.contains('Vallecas')]

Aparece como 'Casco Histórico de Vallecas' en lugar de 'Casco HVallecas', actualizamos esta diferencia en el parking.

In [ ]:
parking_data.loc[parking_data['area'] == 'Casco HVallecas', 'area'] = 'Casco Histórico de Vallecas'

In [ ]:
parking_data[parking_data['title'].str.contains('Altos de Lillo')]

Actualizamos la lista de los barrios que contienen parking.

In [ ]:
filtered_barrios = barrios[barrios['NOMBRE'].isin(parking_areas())]

In [ ]:
# Create the choropleth layer using the filtered GeoJSON data
folium.Choropleth(
    geo_data=filtered_barrios,
    name='choropleth',
    data=total_parking,
    columns=['area', 'general_spaces'],
    key_on='feature.properties.NOMBRE',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2
).add_to(mapa)

folium.GeoJson(filtered_barrios, aliases=['area: '], localize=True, labels=True, parse_html=False).add_to(mapa)

# Display the map
mapa

## Casco HVicalvaro

In [ ]:
parking_data[parking_data['title'].str.contains('San Cipriano')]

Buscamos un barrio en la lista de los barrios que contenga la subcadena 'Vicalvaro'

In [ ]:
barrios[barrios['NOMBRE'].str.contains('Vicálvaro')]

Aparece como 'Casco histórico de Vicálvaro' en lugar de 'Casco HVicalvaro', actualizamos esta diferencia en el parking.

In [ ]:
parking_data.loc[parking_data['area'] == 'Casco HVicalvaro', 'area'] = 'Casco histórico de Vicálvaro'

In [ ]:
parking_data[parking_data['title'].str.contains('San Cipriano')]

Actualizamos la lista de los barrios que contienen parking.

In [ ]:
filtered_barrios = barrios[barrios['NOMBRE'].isin(parking_areas())]

In [ ]:
# Create the choropleth layer using the filtered GeoJSON data
folium.Choropleth(
    geo_data=filtered_barrios,
    name='choropleth',
    data=total_parking,
    columns=['area', 'general_spaces'],
    key_on='feature.properties.NOMBRE',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2
).add_to(mapa)

folium.GeoJson(filtered_barrios, aliases=['area: '], localize=True, labels=True, parse_html=False).add_to(mapa)

# Display the map
mapa

### Estudio descriptivo del conjunto de datos

In [22]:
parking_data

,id,title,locality,district,area,street_address,latitude,longitude,organization,accesibility,public_spaces,resident_spaces,general_spaces
0,11544480,Aparcamiento mixto. Alcántara,MADRID,Salamanca,Goya,CALLE ALCANTARA 26,40.428194,-3.673778,Plazas: 121 públicas y 254 para residentes Ab...,0,121.0,254.0,375.0
1,13470,Aparcamiento mixto. Arquitecto Ribera,MADRID,Centro,Justicia,CALLE BARCELO 2,40.426274,-3.700374,Plazas: 318 públicas y 298 para residentes Ab...,0,318.0,298.0,616.0
2,59060,Aparcamiento mixto. Avenida de América (interc...,MADRID,Chamartin,Prosperidad,AVENIDA AMERICA 9,40.438178,-3.676834,Información sobre accesibilidad: Tiene escal...,0,261.0,395.0,656.0
3,5898885,Aparcamiento mixto. Avenida de Portugal,MADRID,Latina,Puerta Angel,AVENIDA PORTUGAL 155,40.411239,-3.738088,Plazas: 428 públicas y 445 para residentes I...,0,428.0,445.0,873.0
4,13475,Aparcamiento mixto. Brasil,MADRID,Tetuan,Cuatro Caminos,CALLE SAN GERMAN 7,40.456093,-3.693497,Plazas: 226 públicas y 402 para residentes Ab...,0,226.0,402.0,628.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,13614,Aparcamiento para residentes. Vista Alegre,MADRID,Carabanchel,Vista Alegre,AVENIDA PLAZA DE TOROS,40.385867,-3.739736,Plazas: 402 Abierto 24 horas Titularidad: Ay...,0,NaN,NaN,402.0
269,13493,Aparcamiento para residentes. Vital Aza,MADRID,CiudadLineal,Pueblo Nuevo,CALLE VITAL AZA 10 PLANTA -2,40.430749,-3.644351,Plazas: 202 Abierto 24 horas Titularidad: Ay...,0,NaN,NaN,202.0
270,61831,Aparcamiento para residentes. Vía Carpetana,MADRID,Carabanchel,Vista Alegre,CALLE VIA CARPETANA 209,40.388311,-3.744483,Plazas: 309 Abierto 24 horas Titularidad: Ay...,0,NaN,NaN,309.0
271,13514,Aparcamiento para residentes. Ángel Carbajo,MADRID,Tetuan,Castillejos,PLAZA ANGEL CARBAJO 6,40.462270,-3.694398,Plazas: 469 Abierto 24 horas Titularidad: Ay...,0,NaN,NaN,469.0


In [ ]:
public_spaces_by_area = parking_data.groupby("area").sum("public_spaces").sort_values("public_spaces", ascending=False)
public_spaces_by_area

In [ ]:
accsibilidad = parking_data.groupby(["area", "accesibility"]).size().reset_index(name='count').sort_values("count",ascending=False)
accsibilidad

In [24]:
parking_data['district'].unique()

array(['Salamanca', 'Centro', 'Chamartin', 'Latina', 'Tetuan',
       'Moncloa-Aravaca', 'Retiro', 'Moratalaz', 'Chamberi',
       'SanBlas-Canillejas', 'CiudadLineal', 'Fuencarral-ElPardo',
       'Arganzuela', 'VillaDeVallecas', 'Hortaleza', 'PuenteDeVallecas',
       'Carabanchel', 'Villaverde', 'Barajas', 'Vicalvaro', 'Usera'],
      dtype=object)